### ISSM CA3

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import glob
import time

### Dataset attributes
Column 1: Timestamp in seconds  
Column 2: Timestamp in microseconds  
Column 3-15: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S1 (Left Calf)  
Column 16-28: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S2 (Left Thigh)  
Column 29-41: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S3 (Right Calf)  
Column 42-54: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S4 (Right Thigh)  
Column 55-67: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S5 (Back)  
Column 68-80: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S6 (Left Lower Arm)  
Column 81-93: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S7 (Left Upper Arm)  
Column 94-106: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S8 (Right Lower Arm)  
Column 107-119: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S9 (Right Upper Arm)  
Column 120: Label (see activity set) [Walking = 1] [Jogging = 2] [Running = 3]

### Run following cells for the following dataset structure:
*df_walk : all 17 subjects + walking + (accel + gyro + quart)  
*df_jog : all 17 subjects + jogging + (accel + gyro + quart)  
*df_run : all 17 subjects + running + (accel + gyro + quart)  
    
Col[0]: Timestamp in seconds  
Col[1]: Timestamp in microseconds  
Col[67-79]: Sensor S6 (without Mag sensor [74 - 76])  
Col[93-105]: Sensor S8 (without Mag sensor [100 - 103])  
Col[119]: Activity Label

In [27]:
data_dir = '..\dataset'

def readcsvDataset(directory):
    files = glob.glob(directory + '\*.csv')
    df_walk = []
    df_jog = []
    df_run = []
    for file in files:
        df_all = pd.read_csv(file, header=None, delim_whitespace=True)
        df_walk.append(df_all[df_all[119] == 1])
        df_jog.append(df_all[df_all[119] == 2])
        df_run.append(df_all[df_all[119] == 3])
    return pd.concat(df_walk, ignore_index=True), pd.concat(df_jog, ignore_index=True), pd.concat(df_run, ignore_index=True)

def removeOtherSensor(df_walk, df_jog, df_run):
    idx = np.r_[2:67, 74:77, 80:93, 100,103, 106:119]
    return df_walk.drop(df_walk.columns[idx], axis=1), df_jog.drop(df_jog.columns[idx], axis=1), df_run.drop(df_run.columns[idx], axis=1)

In [19]:
%%time
dfw, dfj, dfr = readcsvDataset(data_dir)

Wall time: 34.3 s


In [28]:
dfwalk, dfjog, dfrun = removeOtherSensor(dfw, dfj, dfr)

In [26]:
dfwalk.shape

(45638, 24)

### Run following cells for the following dataset structure: (Running Only)
* df_accel: all 17 subjects + running + accel
* df_jog: all 17 subjects + running + gyro
* df_run: all 17 subjects + running + quart

(45638, 28)

In [95]:
df_jog.shape

(41790, 28)

In [96]:
df_run.shape

(40142, 28)